In [ ]:
import cv2
import numpy as np
from collections import defaultdict

def segment_lines(lines, delta):
    h_lines = []
    v_lines = []
    for line in lines:
        for x1, y1, x2, y2 in line:
            if abs(x2-x1) < delta: # x-values are near; line is vertical
                v_lines.append(line)
            elif abs(y2-y1) < delta: # y-values are near; line is horizontal
                h_lines.append(line)
    return h_lines, v_lines

def find_intersection(line1, line2):
    # extract points
    x1, y1, x2, y2 = line1[0]
    x3, y3, x4, y4 = line2[0]
    # compute determinant
    Px = ((x1*y2 - y1*x2)*(x3-x4) - (x1-x2)*(x3*y4 - y3*x4))/  \
        ((x1-x2)*(y3-y4) - (y1-y2)*(x3-x4))
    Py = ((x1*y2 - y1*x2)*(y3-y4) - (y1-y2)*(x3*y4 - y3*x4))/  \
        ((x1-x2)*(y3-y4) - (y1-y2)*(x3-x4))
    return Px, Py


def cluster_points(points, nclusters):
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    _, _, centers = cv2.kmeans(points, nclusters, None, criteria, 10, cv2.KMEANS_PP_CENTERS)
    return centers


def main():
    
    windowName = "HoughPLineDetector"
    cv2.namedWindow(windowName)
    cap = cv2.VideoCapture(0)
    
    if cap.isOpened():
        ret, frame = cap.read()
    else:
        ret = False


    while ret:
    
        ret, frame = cap.read()  
        
        frame = cv2.GaussianBlur(frame, (7, 7), 1.41) # put camera through gaussian filter to blur it
        
        grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # turn the blurred image to greyscale
        
        edges = cv2.Canny(grey, 25, 75) # run the greyscale footage through canny detector
        
        dilated = cv2.dilate(edges, np.ones((3,3), dtype=np.uint8))
        
        #corners = cv2.goodFeaturesToTrack(grey,1,0.4,50) # find the corners of greyscale image
        
        
        
        #for corner in corners:
           # x,y = corner.flatten()
           # cv2.circle(frame,(x,y),10,(36,255,12),-1)
           # corners = np.int0(corners)
            
       # print (corners)
       # print (x,y)
        
        lines = cv2.HoughLinesP(dilated, 1, np.pi/180, 2, minLineLength=0, maxLineGap=0) # Run Canny through HoughPline detector
        # Draw Probablistic lines on the image
        
        # segment the lines
        delta = 10
        h_lines, v_lines = segment_lines(lines, delta)

        # draw the segmented lines
        for line in h_lines:
            for x1, y1, x2, y2 in line:
                color = [0,0,255] # color hoz lines red
                cv2.line(frame, (x1, y1), (x2, y2), color=color, thickness=3)
        for line in v_lines:
            for x1, y1, x2, y2 in line:
                color = [255,0,0] # color vert lines blue
                cv2.line(frame, (x1, y1), (x2, y2), color=color, thickness=1)

        
        #for line in lines:
           # x1, y1, x2, y2 = line[0]
           # cv2.line(frame, (x1, y1), (x2, y2), (150, 0, 0), 3)
        # Show result
       # print(lines)
            
        cv2.imshow(windowName, frame) #show image result
        
        if cv2.waitKey(1) == 27: # exit on ESC
            break

    cv2.destroyAllWindows()
    cap.release()

if __name__ == "__main__":
    main()
    